First we import all necessary libraries

In [34]:
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go
import chart_studio.plotly as py
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import Dash, Input, Output, callback, dash_table
from plotly.subplots import make_subplots
import dash_bootstrap_components as dbc

# Data cleaning
As a first step the data is read in as a csv file.

In [35]:
data = pd.read_csv("Example_data_masterthesis.csv", sep = ";")

In [36]:
data.head()

,ID,Tested_system,Software_test,Test_result,Execution_date,Test runtime
0,1,System A,Log-in,1,03/02/2022,00:01:34
1,2,System A,Vertrag_a,1,03/02/2022,00:05:41
2,3,System A,Vertrag_b,1,03/02/2022,00:09:48
3,4,System A,Vertrag_c,1,03/02/2022,00:03:55
4,5,System A,Vertrag_d,1,03/02/2022,00:02:02


As we want to be able to add various different data, we change the column names, so that we can assure they stay the same. The old name of the columns will vary from data to data and will need to be adapted. 

In [37]:
data = data.rename(columns={"Tested_system":"system", "Software_test":"testname", "Test_result":"result", "Execution_date":"date", "Test runtime":"runtime"})

After the data is read in, we want to check, if all data types are set correct. As this is not the case, we set the correct types

In [38]:
data.dtypes

ID           int64
system      object
testname    object
result       int64
date        object
runtime     object
dtype: object

In [39]:
data = data.astype({"result": "bool", "date":"datetime64"})
data["runtime"] = pd.to_datetime(data["runtime"], format= "%H:%M:%S").dt.time
data["date"] = pd.to_datetime(data["date"], format= "%D:%M:%Y").dt.date
data.head()

,ID,system,testname,result,date,runtime
0,1,System A,Log-in,True,2022-03-02,00:01:34
1,2,System A,Vertrag_a,True,2022-03-02,00:05:41
2,3,System A,Vertrag_b,True,2022-03-02,00:09:48
3,4,System A,Vertrag_c,True,2022-03-02,00:03:55
4,5,System A,Vertrag_d,True,2022-03-02,00:02:02


Now we check if there are any null values

In [40]:
data.isna().any()

ID          False
system      False
testname    False
result      False
date        False
runtime     False
dtype: bool

Here we set a column as an id.

In [41]:
data['id'] = data['ID']
data.set_index('id', inplace=True, drop=False)
print(data.columns)

Index(['ID', 'system', 'testname', 'result', 'date', 'runtime', 'id'], dtype='object')


In order to make the table more readable, we use symbols for passed/failed tests

In [42]:
data['result'].loc[(data['result'] == True)] = '🔵 Passed'
data['result'].loc[(data['result'] == False)] = '🔴 Failed'
data.head()

c:\Users\gaert\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ID,system,testname,result,date,runtime,id
id,,,,,,,
1,1,System A,Log-in,🔵 Passed,2022-03-02,00:01:34,1
2,2,System A,Vertrag_a,🔵 Passed,2022-03-02,00:05:41,2
3,3,System A,Vertrag_b,🔵 Passed,2022-03-02,00:09:48,3
4,4,System A,Vertrag_c,🔵 Passed,2022-03-02,00:03:55,4
5,5,System A,Vertrag_d,🔵 Passed,2022-03-02,00:02:02,5


In [43]:
table_data = data[["system", "testname", "result", "date", "runtime"]]
table_data.head()

,system,testname,result,date,runtime
id,,,,,
1,System A,Log-in,🔵 Passed,2022-03-02,00:01:34
2,System A,Vertrag_a,🔵 Passed,2022-03-02,00:05:41
3,System A,Vertrag_b,🔵 Passed,2022-03-02,00:09:48
4,System A,Vertrag_c,🔵 Passed,2022-03-02,00:03:55
5,System A,Vertrag_d,🔵 Passed,2022-03-02,00:02:02


In [44]:
y = data.loc[(data['result'] == '🔵 Passed')]
passed_per_day=[]
grouped_per_day = []
group_size = 3
for j in data['system'].unique():
    x = y.loc[(data['system']) == j]
    z = data.loc[(data['system']) == j]
    for i in data['date'].unique():
        passed_per_day.append((len(x.loc[data['date'] == i]))/len(z.loc[data['date'] == i])*100)
        passed_per_day.append(str(i))
        passed_per_day.append(str(j))

for i in range (0, len(passed_per_day), group_size):
    grouped_per_day.append(passed_per_day[i:i+group_size]) 

data_passed_percentage = pd.DataFrame(grouped_per_day, columns = ['passed%', 'date', 'system'])
data_passed_percentage



,passed%,date,system
0,100.0,2022-03-02,System A
1,60.0,2022-04-02,System A
2,60.0,2022-03-02,System B
3,80.0,2022-04-02,System B


In [45]:
y = data.loc[(data['result'] == '🔵 Passed')]
x = y.loc[(data['system']) == 'System A']
x.loc[data['date'] == i]

,ID,system,testname,result,date,runtime,id
id,,,,,,,


# Dashboard creation
Next we start building the dashboard.

In [46]:
app = Dash(__name__)

In [47]:
app.layout = html.Div([

    html.Div([
        html.H1("Software test results", style={'text-align': 'center'}),
    ], style={
        'padding-bottom': '1%',
        'padding-top': '1%',
        #'padding-left' : '3%',
        "background" : "#EAEAEA",
        "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
        }),
    html.Br(),
    html.Br(),
    html.Div(children=[

    # Dropdown menue
        html.Div([
            html.Br(),
            dcc.Dropdown(
                id='dropdown', multi=True, clearable=False,
                options=[{'label': i, 'value': i} for i in data["system"].unique()],
                style = {
                    "width": "100%",
                    "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
                    },
                placeholder="Select the systems you want to see details on. Currently all data is visualized.",
                #value= data["system"].unique()[0]
                        ),
        ], style = {
            'display': 'inline-block', 
            'padding-right': '5%',
            'padding-bottom' : '5%',
            'margin-left': '0',
            'margin-right': 'auto',
            'width' : '100%',
        }),

        # Datatable
        html.Div([
            html.Br(),
            html.H3('Details on conducted tests: ' , style = {'text-align': 'center'}),
            html.Br(),
            dash_table.DataTable(
                data.to_dict('records'), 
                id="datatable",
                columns= [
                    {"name": i, "id": i} 
                    if i == "testname" or i == "result" or i == "date"
                    else {"name": i, "id": i}
                    for i in table_data.columns],
                style_cell={
                    'textAlign': 'left',
                    'padding': '5px',
                    'font-family': 'Arial, Helvetica, sans-serif',
                    },
                style_header = {
                    'fontWeight': 'bold',
                    'border' : '1px solid black',
                    'backgroundColor' : 'rgb(224, 224, 224)',
                    'font-family': 'Arial, Helvetica, sans-serif',
                },
                style_data_conditional=[
                    {
                        'if' : {'row_index' : 'odd'},
                        'backgroundColor' : 'rgb(242, 243, 245)'
                    }
                ],
                sort_action="native",
                page_action='native',
                page_size=20, 
                style_table={'height': 1500, 'overflowY': 'auto'},
                fixed_rows={'headers': True},
                filter_action="native",
                 ),
        ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            'display': 'float',
            'float' : 'right',
            'padding-left' : '7%',
            'padding-right' : '2%',
            'padding-bottom' :'3%',
            "width": "96%",
            "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
            }),      
        html.Br(),

    # Dropdowns for the Boxplots

        html.Div([html.P(),
            html.H4('Date 1'),
                dcc.Dropdown(id='boxplot_choice1', multi=False, clearable=False,
                        options=[{'label': i, 'value': i} for i in
                        data["date"].unique()],
                        value= data["system"].unique()[0],
                        style = {"box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"},
                        placeholder="Select the first date you want to compare",
                                ),
        ], style={
            "width": "46%", 
            'display': 'inline-block',
            'padding-bottom' : '5%',
            }),

        html.Div([html.P(),
            html.H4('Date 2'),
                dcc.Dropdown(id='boxplot_choice2', multi=False, clearable=False,
                        options=[{'label': i, 'value': i} for i in
                        data["date"].unique()],
                        value= data["system"].unique()[0],
                        style = {"box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"},
                        placeholder="Select the second date you want to compare",
                                ),
                            
        ], style={
            "width": "46%", 
            'display': 'inline-block', 
            'padding-left': '5%',
            'padding-bottom' : '3%',
            'margin-left': '0',
            'margin-right': 'auto',
            }),

     # Boxplots
     html.Div([
        html.H3('Comparison of runtime per selected date ' , style = {'text-align': 'center'}),
        html.P('Please select the two dates you want to compare with each other ' , style = {'text-align': 'center'}),
        dcc.Graph(id='boxplot'),
     ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            'display': 'float',
            'float' : 'right',
            'padding-left' : '7%',
            'padding-right' : '2%',
            'padding-bottom' :'3%',
            "width": "96%",
            "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
            }
        ),

    ], style={
        "width": "52%",
        'font-family': 'Arial, Helvetica, sans-serif',
        'padding-left': '4%', 
        'display': 'float', 
        'float' : 'left',
        'padding-top': '1%',
        'margin-left': '0',
        'margin-right': 'auto',
        }),

   # Visualize number of conducted tests
    html.Div(children=[

        html.Div([
            html.Br(),
            html.H3('Number of conducted tests: ' , style = {'text-align': 'center'}),
            html.P(
            id='test_number',
            style = {'text-align': 'center', 'font-size' : '3.5em'},
            )
        ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
            }
        ),
        html.Br(),
        html.Br(),
    # Piechart
        html.Div([
            html.Br(),
            html.H3('Percentage of passed and failed tests: ' , style = {'text-align': 'center'}),
            dcc.Graph(id='piechart', style={"height": "10%"}),
        ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            "box-shadow": "0 4px 8px 0 rgba(0, 0, 0, 0.2), 0 6px 20px 0 rgba(0, 0, 0, 0.19)"
            }
        ),
        html.Br(),
        html.Br(),
    # Linechart
        html.Div([
            html.Br(),
            html.H3('Passed test rate over time: ' , style = {'text-align': 'center'}),
            dcc.Graph(id='linechart')
        ], style = {
            'border': 'solid black 1.5px',
            'background' : '#5a6c8a',
            
            },
        ),

    ],style={
        "width": "40%", 
        'display': 'float',
        'float' : 'right',
        'font-family': 'Arial, Helvetica, sans-serif',
        'padding-top': '1%',
        'margin-left': 'auto',
        'margin-right': '0',
        #"background" : "#5a6c8a",
        #'border' : 'solid black 2px'
    }),
    
], style={
    "padding": "10px",
    'font-family': 'Arial, Helvetica, sans-serif',
})
 

In [48]:
@app.callback(
    [Output(component_id='test_number', component_property='children'),
    Output(component_id='boxplot', component_property='figure'),
    Output(component_id='piechart', component_property='figure'),
    Output(component_id='linechart', component_property='figure')],
    [Input(component_id='dropdown', component_property='value'),
    Input(component_id='boxplot_choice1', component_property='value'),
    Input(component_id='boxplot_choice2', component_property='value'),
    Input(component_id='datatable', component_property="derived_virtual_data")]
)

def update_data(option_slctd, date_choice1, date_choice2, all_rows_data):

    # Defining the three base data tables on which the graphs are built upon
    dff = pd.DataFrame(all_rows_data)
    #print('Data across all pages pre or post filtering: {}'.format(option_slctd)),
    #print('Data across all pages pre or post filtering: {}'.format(len(option_slctd)))

    if option_slctd == None:
        data_filtered = dff
    else:
        if len(option_slctd) != 0:
            data_filtered = dff.loc[dff['system'].isin(option_slctd)]
        else:
            data_filtered = dff
    
    boxplot_data = data_filtered
    boxplot_data['runtime'] = pd.to_datetime(data['runtime'], format= "%H:%M:%S", infer_datetime_format=True)

    dff_sub= data_filtered.groupby('result').size().reset_index(name='count')

    # Calculations for the % of passed tests per day 
    y = data_filtered.loc[(data_filtered['result'] == '🔵 Passed')]
    passed_per_day=[]
    grouped_per_day = []
    group_size = 2
    for i in data_filtered['date'].unique():
        passed_per_day.append((len(y.loc[data_filtered['date'] == i]))/len(data_filtered.loc[data_filtered['date'] == i])*100)
        passed_per_day.append(str(i))

    for i in range (0, len(passed_per_day), group_size):
        grouped_per_day.append(passed_per_day[i:i+group_size]) 

    data_passed_percentage = pd.DataFrame(grouped_per_day, columns = ['passed%', 'date'])
    data_passed_percentage

    number_of_tests = len(data_filtered)

    # Building the boxplots
    trace0 = go.Box(
        y=boxplot_data.loc[boxplot_data['date'] == str(date_choice1)]['runtime'].sort_values(), 
        name = date_choice1,
        marker = dict(
        color = 'rgb(44, 39, 117)'
        ),
        hoverinfo='none'
    )
    
    trace1 = go.Box(
        y= boxplot_data.loc[boxplot_data['date'] == str(date_choice2)]['runtime'].sort_values(),
        name = date_choice2,
        marker = dict(
        color = 'rgb(50, 173, 155)'
        ),
        hoverinfo='none'
    )
    data1 = [trace0,trace1]
    layout = go.Layout(  
        template='ggplot2',
        #labels = [date_choice1, date_choice2],
        #text = ["textA", "TextB", "TextC", "TextD"],
        #hovertemplate = "%{label}: <br>Runtime distribution: %{max}"
    )

    box_plot = go.Figure(data=data1,layout=layout)
    box_plot.update_layout(paper_bgcolor="#5a6c8a")

    # Building the piechart
    pie_chart=px.pie(
            data_frame=dff_sub,
            names='result',
            values='count',
            hole=.3,
            labels={'result':'count'}
            )
    pie_chart.update_layout(paper_bgcolor="#5a6c8a")
    # Building the line chart
    line_chart = px.line(
            data_frame=data_passed_percentage,
            x='date',
            y='passed%',
            #labels={'countriesAndTerritories':'Countries', 'dateRep':'date'},
            )
    line_chart.update_layout(hovermode="x unified", paper_bgcolor="#5a6c8a")
    line_chart.update_xaxes(rangeslider_visible=True)
    line_chart.update_traces(mode="markers+lines")

    return (number_of_tests, box_plot, pie_chart, line_chart)

In [49]:
if __name__ == '__main__':
    app.run_server(debug=False)
    

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jun/2022 13:33:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 13:33:27] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 13:33:27] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 13:33:27] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Jun/2022 13:33:27] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Jun/2022 13:33:27] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Jun/2022 13:33:27] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Jun/2022 13:33:27] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\Users\gaert\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispatch
    response.set_d

127.0.0.1 - - [27/Jun/2022 13:33:27] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/Jun/2022 13:33:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:01:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:01:05] "POST /_dash-update-component HTTP/1.1" 200 -
C:\Users\gaert\AppData\Local\Temp/ipykernel_26444/198920061.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

127.0.0.1 - - [27/Jun/2022 14:01:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:01:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:01:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:02:18] "POST /_dash-update-compo

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\gaert\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\Users\gaert\anaconda3\lib\site-packages\dash\dash.py", line 1345, in dispatch
    response.set_d

127.0.0.1 - - [27/Jun/2022 14:02:23] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [27/Jun/2022 14:02:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:02:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:02:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:03:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:03:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:03:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2022 14:03:47] "POST /_dash-update-component HTTP/1.1" 200 -
